This notbook is used to split the ingredients into multiple columns for easy analysis.

In [14]:
# import dependencies
import pandas as pd
import numpy as np
import re

In [5]:
# import the raw csv
recipes_df_raw = pd.read_csv('technical_list.csv')
recipes_df_raw.head

<bound method NDFrame.head of                                              title           judge  \
0           Liam Charles’s Yorkshire Pudding Wraps    Liam Charles   
1      Rav Gill’s Churros with Banana Butterscotch    Ravneet Gill   
2                 Liam Charles’s Mega-Choccy Stack    Liam Charles   
3                        Rav Gill’s Tiger Pretzels    Ravneet Gill   
4    Liam Charles’s Chocolate Marshmallow Biscuits    Liam Charles   
..                                             ...             ...   
121                Mary Berry’s Hazelnut Dacquoise  Paul Hollywood   
122                       Mary Berry’s Religieuses  Paul Hollywood   
123              Paul Hollywood’s Apricot Couronne  Paul Hollywood   
124      Mary Berry’s Tuiles with Chocolate Mousse  Paul Hollywood   
125                  Mary Berry’s Charlotte Royale  Paul Hollywood   

            makes              difficulty hands_on_time baking_time  \
0               2             Needs skill       25 mins   

In [6]:
# Make a function to remove p tags from the list
def remove_tags(ingredients):
        clean_ingredients = ingredients.replace('<p>','').replace('</p>','')
        return clean_ingredients

In [12]:
ingredients = recipes_df_raw['ingredients']
recipes_no_tags = recipes_df_raw.copy()
recipes_no_tags['ingredients_clean'] = ''
for i in range(len(ingredients)):
    ingredient_list = ingredients[i]
    clean_list = remove_tags(ingredient_list)
    recipes_no_tags['ingredients_clean'][i] = clean_list
print(recipes_no_tags['ingredients_clean'])

0      [100g plain flour, 2 large eggs, 200ml whole m...
1      [20g caster sugar, 75g unsalted butter, cubed ...
2      l olive oil, 1 tsp vanilla paste, 150g ca...
3      l warm water , 4g active dried yeast , 10...
4      [50g plain flour, plus extra for dusting, ½ ts...
                             ...                        
121    [250g blanched (skinned) whole hazelnuts, 300g...
122    [60g unsalted butter, cubed, 150ml water, 75g ...
123    [250g strong white bread flour, 5g salt, 7g sa...
124    [200g unsalted butter, softened, 180g icing su...
125    [400g strawberries, hulled and chopped, 500g j...
Name: ingredients_clean, Length: 126, dtype: object


In [17]:
def split_ingredient(ingredient):
    quantity = re.findall('\d+', ingredient)
    print(quantity)

In [18]:
test_ingredient = '100g Can Sugar'
split_ingredient(test_ingredient)

['100']
